In [1]:
import faiss
import numpy as np
import torch
vectors = np.load('embeddings.npy')

In [2]:
print(vectors.shape)

(300, 768)


In [3]:
vectors=vectors.astype(np.float32)
dimention=vectors.shape[1]
index=faiss.IndexFlatL2(dimention)
index.add(vectors)
print("Number of vectors in the index:", index.ntotal)

Number of vectors in the index: 300


In [ ]:
def retrive(info,index, k=5):
    info=np.array(info).reshape(1,-1).astype(np.float32)
    distance,indices=index.search(info,k)
    return indices,distance

In [ ]:
tools=[{
    "type": "function",
    "function": {
        "name": "retrive",
        "description": "Cung cấp những tài liệu/trích dẫn liên quan đến thông tin mà bạn tìm kiếm về  luat phap ",
        "parameters": {
            "type": "object",
            "properties": {
                "info": {
                    "type": "string",
                    "description": "Thông tin/lĩnh vực ve phap luat can tim kiem, vi du:'luat nhan qua'",
                },
                'index':{
                    "type": "object",
                    "description":"index can trich xuat thong tin ra",
                },
                "k":{
                    'type':"constant",
                    'description':"so luong vector lay ra tu trong index",
                }
            },
            "required": ["info","index","k"],
        },
    },
}]


In [ ]:
import json 
import ollama
import asyncio 
async def run(model,user_input):
    client=ollama.AsyncClient()
    messages=[
        {
            'role':'user',
            'content':user_input,
        }
    ]
    response=await client.chat(
        model=model,
        messages=messages,
        tools=tools,
    )

/tmp/ipykernel_6667/1647321461.py:3: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = OllamaFunctions(model="llama3.1", format="json")


In [13]:
llm = llm.bind_tools(tools, function_call={"name": "retrive"})